In [1]:
import csv
import os
import pandas as pd
import numpy as np
import copy
import time
import pickle
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Activation, GRU
from sklearn.preprocessing import normalize, LabelEncoder, MinMaxScaler

import sklearn.model_selection as model_selection
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, det_curve
import matplotlib.pyplot as plt

2022-06-07 21:36:58.105608: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-07 21:36:58.105648: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [22]:
# usefull fucntions
def remove_nan_values(sample_array:np.array) -> np.array:
    for sample_index, i in enumerate(sample_array):
        for value_index, j in enumerate(i):
            if j==' ':
               sample_array[sample_index][value_index] = 0.0
            elif type(j) == str:
                sample_array[sample_index][value_index] = int(j)
            elif np.isnan(j):
               sample_array[sample_index][value_index] = 0.0
    return sample_array

def remove_1d_array_nan_values(sample_array: np.array, attack_categories: dict) -> np.array:
    for value_index, j in enumerate(sample_array):
        if j==' ':
           sample_array[value_index] = 0.0
        elif type(j) == str:
            try:
                sample_array[value_index] = int(j)
            except:
                sample_array[value_index] = attack_categories[j]
        elif np.isnan(j):
           sample_array[value_index] = 0.0
    return sample_array

def save_model(model, name, prefix) -> None:
    """Function responsible for saving trained model. It must be called
    after defining, training and predict.

    :param: None
    :return: None
    """

    filename = f"{name}_{prefix}.sav"
    pickle.dump(model, open(filename, 'wb'))
    
def load_model(filepath):
    """Function responsible for load model.

    :param: None
    :return: None
    """
    return pickle.load(open(filepath, 'rb'))

In [23]:
dataset_testing_path = '/home/mkubita/Pulpit/Praca Magisterska/Zbiory danych/UNSW-NB15/UNSW-NB15 - CSV Files/Splitted_Dataset/UNSW_NB15_testing-set.csv'
dataset_traning_path = '/home/mkubita/Pulpit/Praca Magisterska/Zbiory danych/UNSW-NB15/UNSW-NB15 - CSV Files/Splitted_Dataset/UNSW_NB15_training-set.csv'


dataset_features_path = '/home/mkubita/Pulpit/Praca Magisterska/Zbiory danych/UNSW-NB15/UNSW-NB15 - CSV Files/NUSW-NB15_features.csv'

features_columns = pd.read_csv(dataset_features_path, sep=",", encoding='cp1252')
features_list = list(features_columns['Name'])

In [24]:
# load data to pandas DataFrame
df_training = pd.read_csv(dataset_traning_path, index_col=None, header=0)
df_training = df_training.drop(df_training.columns[0], axis=1)

df_testing = pd.read_csv(dataset_testing_path, index_col=None, header=0)
df_testing = df_testing.drop(df_testing.columns[0], axis=1)

In [25]:
# features list
features_list = list(df_training.columns)
features_list.remove('attack_cat')
features_list.remove('label')

In [26]:
# categorical values
categorical_features = ['state', 'proto', 'service']

for categorical_feature in categorical_features:
    test_list = list(df_testing[features_list][categorical_feature].unique())

    label_encoding = {name: index for index, name in enumerate(list(df_training[features_list][categorical_feature].unique()))}
    for i in test_list:
        if i not in label_encoding.keys():
            label_encoding[i] = max(label_encoding.values()) + 1
    
    
    for name, value in label_encoding.items():
        df_training[features_list]= df_training[features_list].replace(name, value)
        df_testing[features_list]= df_testing[features_list].replace(name, value)


### Binary classification

In [27]:
# prepare data for classification
x_train = df_training[features_list].to_numpy()
y_train = df_training['label'].to_numpy()

x_test = df_testing[features_list].to_numpy()
y_test = df_testing['label'].to_numpy()

# remove nan values
x_train = remove_nan_values(x_train)
x_test = remove_nan_values(x_test)

In [28]:
df_training

dur  proto  service  state  spkts  dpkts  sbytes  dbytes  \
0       0.121478      0        0      0      6      4     258     172   
1       0.649902      0        0      0     14     38     734   42014   
2       1.623129      0        0      0      8     16     364   13186   
3       1.681642      0        1      0     12     12     628     770   
4       0.449454      0        0      0     10      6     534     268   
...          ...    ...      ...    ...    ...    ...     ...     ...   
175336  0.000009      1        6      1      2      0     114       0   
175337  0.505762      0        0      0     10      8     620     354   
175338  0.000009      1        6      1      2      0     114       0   
175339  0.000009      1        6      1      2      0     114       0   
175340  0.000009      1        6      1      2      0     114       0   

                 rate  sttl  ...  ct_dst_sport_ltm  ct_dst_src_ltm  \
0           74.087490   252  ...                 1               1   
1           78.473372    62  ...                 1               2   
2           14.170161    62  ...                 1               3   
3           13.677108    62  ...                 1               3   
4           33.373826   254  ...                 1              40   
...               ...   ...  ...               ...             ...   
175336  111111.107200   254  ...                13              24   
175337      33.612649   254  ...                 1               2   
175338  111111.107200   254  ...                 3              13   
175339  111111.107200   254  ...                14              30   
175340  111111.107200   254  ...                16              30   

        is_ftp_login  ct_ftp_cmd  ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  \
0                  0           0                 0           1           1   
1                  0           0                 0           1           6   
2                  0           0                 0           2           6   
3                  1           1                 0           2           1   
4                  0           0                 0           2          39   
...              ...         ...               ...         ...         ...   
175336             0           0                 0          24          24   
175337             0           0                 0           1           1   
175338             0           0                 0           3          12   
175339             0           0                 0          30          30   
175340             0           0                 0          30          30   

        is_sm_ips_ports  attack_cat  label  
0                     0      Normal      0  
1                     0      Normal      0  
2                     0      Normal      0  
3                     0      Normal      0  
4                     0      Normal      0  
...                 ...         ...    ...  
175336                0     Generic      1  
175337                0   Shellcode      1  
175338                0     Generic      1  
175339                0     Generic      1  
175340                0     Generic      1  

[175341 rows x 44 columns]

In [9]:
# normalization min max
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [18]:
model = Sequential()
# model.add(Embedding(64, 42, input_length=64))

# !!! Play with those! try and get better results!
#model.add(SimpleRNN(128))  
#model.add(GRU(128))  
model.add(LSTM(300, activation="tanh", return_sequences = True, input_shape = (42, 1)))
model.add(Dropout(0.4))

model.add(LSTM(200, activation="tanh", return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(100, activation="tanh", return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(80, activation="tanh"))
model.add(Dropout(0.4))

model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(x_train, y_train, batch_size=64, 
          epochs=20, validation_data=(x_test, y_test))


Epoch 1/50
2740/2740 [==============================] - 616s 223ms/step - loss: 0.2084 - val_loss: 0.3556
Epoch 2/50
2740/2740 [==============================] - 632s 231ms/step - loss: 0.3367 - val_loss: 0.6172
Epoch 3/50
2740/2740 [==============================] - 634s 231ms/step - loss: 0.5347 - val_loss: 0.5831
Epoch 4/50
2740/2740 [==============================] - 649s 237ms/step - loss: 0.5297 - val_loss: 0.5658
Epoch 5/50
2740/2740 [==============================] - 660s 241ms/step - loss: 0.5023 - val_loss: 0.5439
Epoch 6/50
2740/2740 [==============================] - 639s 233ms/step - loss: 0.5022 - val_loss: 0.5616
Epoch 7/50
2740/2740 [==============================] - 679s 248ms/step - loss: 0.4446 - val_loss: 0.3810
Epoch 8/50
2740/2740 [==============================] - 656s 239ms/step - loss: 0.2263 - val_loss: 0.4000
Epoch 9/50
2740/2740 [==============================] - 663s 242ms/step - loss: 0.1962 - val_loss: 0.3669
Epoch 10/50
2740/2740 [=======================

TypeError: cannot unpack non-iterable float object

In [45]:
prediction_lstm = model.predict(x_test)

In [55]:
a = prediction_lstm.flatten()
a

array([9.5346922e-01, 8.9072037e-01, 8.7229699e-01, ..., 3.8040637e-06,
       3.8040637e-06, 9.1604716e-01], dtype=float32)

In [58]:
pre2 = [int(round(i,0)) for i in a]

In [41]:
model.evaluate(x_test, y_test, batch_size=64)

1287/1287 [==============================] - 112s 87ms/step - loss: 0.2743


0.27425017952919006

In [59]:
accuracy = accuracy_score(y_test, pre2)
precision = precision_score(y_test, pre2, average='binary')
recall = recall_score(y_test, pre2, average='binary')
f1_ = f1_score(y_test, pre2, average='binary')

print(accuracy, precision, recall, f1_)

0.8666375163970267 0.8125068228958189 0.9851098561722403 0.8905218657147985


In [21]:
save_model(model, 'LSTM', 'BIN_ALL')

2022-05-31 09:26:42.289700: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://5fe980b4-f6a6-4972-a3f2-3425650c5112/assets


INFO:tensorflow:Assets written to: ram://5fe980b4-f6a6-4972-a3f2-3425650c5112/assets


In [10]:
model_GRU = Sequential()

model_GRU.add(GRU(300, activation="tanh", return_sequences = True, input_shape = (42, 1)))
model_GRU.add(Dropout(0.4))

model_GRU.add(GRU(200, activation="tanh", return_sequences = True))
model_GRU.add(Dropout(0.4))

model_GRU.add(GRU(100, activation="tanh", return_sequences = True))
model_GRU.add(Dropout(0.4))

model_GRU.add(GRU(80, activation="tanh"))
model_GRU.add(Dropout(0.4))

model_GRU.add(Dense(1))
model_GRU.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model_GRU.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-05-31 18:03:47.152836: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 18:03:47.153495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 18:03:47.153612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-31 18:03:47.153719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-31 18:03:47.176446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [11]:
model_GRU.fit(x_train, y_train, batch_size=64, 
          epochs=20, validation_data=(x_test, y_test))


Epoch 1/20
2740/2740 [==============================] - 594s 214ms/step - loss: 0.2007 - accuracy: 0.9150 - val_loss: 0.3334 - val_accuracy: 0.8394
Epoch 2/20
2740/2740 [==============================] - 582s 213ms/step - loss: 0.1382 - accuracy: 0.9348 - val_loss: 0.3476 - val_accuracy: 0.8381
Epoch 3/20
2740/2740 [==============================] - 586s 214ms/step - loss: 0.1340 - accuracy: 0.9363 - val_loss: 0.3162 - val_accuracy: 0.8452
Epoch 4/20
2740/2740 [==============================] - 528s 193ms/step - loss: 0.1296 - accuracy: 0.9370 - val_loss: 0.2601 - val_accuracy: 0.8683
Epoch 5/20
2740/2740 [==============================] - 533s 195ms/step - loss: 0.1253 - accuracy: 0.9390 - val_loss: 0.2830 - val_accuracy: 0.8448
Epoch 6/20
2740/2740 [==============================] - 561s 205ms/step - loss: 0.1235 - accuracy: 0.9399 - val_loss: 0.3207 - val_accuracy: 0.8271
Epoch 7/20
2740/2740 [==============================] - 560s 205ms/step - loss: 0.1207 - accuracy: 0.9408 - val_

In [ ]:
# score2 = model_GRU.evaluate(x_test, y_test, batch_size=100)
# print('Test score:', score2)
# # print('Test accuracy:', acc)

In [17]:
prediction_gru_bin = model_GRU.predict(x_test)

In [18]:
pre3 = [int(round(i,0)) for i in prediction_gru_bin.flatten()]

In [19]:
accuracy2 = accuracy_score(y_test, pre3)
precision2 = precision_score(y_test, pre3, average='binary')
recall2 = recall_score(y_test, pre3, average='binary')
f1_2 = f1_score(y_test, pre3, average='binary')

print(accuracy2, precision2, recall2, f1_2)

0.843523781761648 0.7822093892957158 0.9920144710138533 0.8747070208027387


In [12]:
save_model(model_GRU, 'GRU', 'BIN_ALL')

2022-05-31 21:17:05.420749: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://7e0533cf-6341-4efa-a07d-fc69341bd15c/assets


INFO:tensorflow:Assets written to: ram://7e0533cf-6341-4efa-a07d-fc69341bd15c/assets


In [10]:
chosen_features = ['sttl', 'ct_state_ttl', 'sload', 'rate', 'dload', 'dttl', 'sbytes', 'ct_srv_dst', 'smean', 
                  'dmean', 'dbytes', 'ackdat', 'ct_dst_src_ltm', 'ct_srv_src', 'dur', 'tcprtt', 'synack',
                  'dinpkt', 'sinpkt', 'dpkts']


In [11]:
# prepare data for classification
x_train_reduced_bin = df_training[features_list][chosen_features].to_numpy()
y_train_reduced_bin = df_training['label'].to_numpy()

x_test_reduced_bin = df_testing[features_list][chosen_features].to_numpy()
y_test_reduced_bin = df_testing['label'].to_numpy()

# remove nan values
x_train_reduced_bin = remove_nan_values(x_train_reduced_bin)
x_test_reduced_bin = remove_nan_values(x_test_reduced_bin)

In [12]:
# normalization min max
scaler = MinMaxScaler()
x_train_reduced_bin = scaler.fit_transform(x_train_reduced_bin)
x_test_reduced_bin = scaler.fit_transform(x_test_reduced_bin)

In [24]:
model_LSTM_BIN_reduced = Sequential()

model_LSTM_BIN_reduced.add(LSTM(300, activation="tanh", return_sequences = True, input_shape = (20, 1)))
model_LSTM_BIN_reduced.add(Dropout(0.4))

model_LSTM_BIN_reduced.add(LSTM(200, activation="tanh", return_sequences = True))
model_LSTM_BIN_reduced.add(Dropout(0.4))

model_LSTM_BIN_reduced.add(LSTM(100, activation="tanh", return_sequences = True))
model_LSTM_BIN_reduced.add(Dropout(0.4))

model_LSTM_BIN_reduced.add(LSTM(80, activation="tanh"))
model_LSTM_BIN_reduced.add(Dropout(0.4))

model_LSTM_BIN_reduced.add(Dense(1))
model_LSTM_BIN_reduced.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model_LSTM_BIN_reduced.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model_LSTM_BIN_reduced.fit(x_train_reduced_bin, y_train_reduced_bin, batch_size=64, 
          epochs=20, validation_data=(x_test_reduced_bin, y_test_reduced_bin))

Epoch 1/20
2740/2740 [==============================] - 318s 114ms/step - loss: 0.4994 - accuracy: 0.7750 - val_loss: 0.6366 - val_accuracy: 0.7660
Epoch 2/20
2740/2740 [==============================] - 317s 116ms/step - loss: 0.2662 - accuracy: 0.9200 - val_loss: 0.5886 - val_accuracy: 0.7660
Epoch 3/20
2740/2740 [==============================] - 318s 116ms/step - loss: 0.2628 - accuracy: 0.9206 - val_loss: 0.6184 - val_accuracy: 0.7660
Epoch 4/20
2740/2740 [==============================] - 318s 116ms/step - loss: 0.3275 - accuracy: 0.8880 - val_loss: 0.5937 - val_accuracy: 0.7660
Epoch 5/20
2740/2740 [==============================] - 319s 116ms/step - loss: 0.2651 - accuracy: 0.9182 - val_loss: 0.5829 - val_accuracy: 0.7660
Epoch 6/20
2740/2740 [==============================] - 318s 116ms/step - loss: 0.2626 - accuracy: 0.9196 - val_loss: 0.5468 - val_accuracy: 0.7885
Epoch 7/20
2740/2740 [==============================] - 318s 116ms/step - loss: 0.2612 - accuracy: 0.9202 - val_

In [29]:
lstm_reduced_bin = model_LSTM_BIN_reduced.predict(x_test_reduced_bin)

In [30]:
pre4 = [int(round(i,0)) for i in lstm_reduced_bin.flatten()]

In [32]:
accuracy3 = accuracy_score(y_test_reduced_bin, pre4)
precision3 = precision_score(y_test_reduced_bin, pre4, average='binary')
recall3 = recall_score(y_test_reduced_bin, pre4, average='binary')
f1_3 = f1_score(y_test_reduced_bin, pre4, average='binary')

print(accuracy3, precision3, recall3, f1_3)

0.7884540640334257 0.7234451292723926 0.9968675549280861 0.8384277855598951


In [26]:
save_model(model_LSTM_BIN_reduced, 'LSTM', 'BIN_REDUCED')

INFO:tensorflow:Assets written to: ram://5bdd7e7c-ec85-4f17-a4d7-52f8c64ebf2b/assets


INFO:tensorflow:Assets written to: ram://5bdd7e7c-ec85-4f17-a4d7-52f8c64ebf2b/assets


In [13]:
model_GRU_REDUCED = Sequential()

model_GRU_REDUCED.add(GRU(300, activation="tanh", return_sequences = True, input_shape = (20, 1)))
model_GRU_REDUCED.add(Dropout(0.4))

model_GRU_REDUCED.add(GRU(200, activation="tanh", return_sequences = True))
model_GRU_REDUCED.add(Dropout(0.4))

model_GRU_REDUCED.add(GRU(100, activation="tanh", return_sequences = True))
model_GRU_REDUCED.add(Dropout(0.4))

model_GRU_REDUCED.add(GRU(80, activation="tanh"))
model_GRU_REDUCED.add(Dropout(0.4))

model_GRU_REDUCED.add(Dense(1))
model_GRU_REDUCED.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model_GRU_REDUCED.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-06-07 21:39:56.428147: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 21:39:56.429081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-07 21:39:56.429192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-07 21:39:56.429283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-07 21:39:56.431512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [34]:
model_GRU_REDUCED.fit(x_train_reduced_bin, y_train_reduced_bin, batch_size=64, 
          epochs=20, validation_data=(x_test_reduced_bin, y_test_reduced_bin))

Epoch 1/20
2740/2740 [==============================] - 265s 95ms/step - loss: 0.2842 - accuracy: 0.9100 - val_loss: 0.6143 - val_accuracy: 0.7660
Epoch 2/20
2740/2740 [==============================] - 300s 110ms/step - loss: 0.2632 - accuracy: 0.9205 - val_loss: 0.5945 - val_accuracy: 0.7660
Epoch 3/20
2740/2740 [==============================] - 291s 106ms/step - loss: 0.2604 - accuracy: 0.9204 - val_loss: 0.5935 - val_accuracy: 0.7660
Epoch 4/20
2740/2740 [==============================] - 297s 108ms/step - loss: 0.2589 - accuracy: 0.9203 - val_loss: 0.6104 - val_accuracy: 0.7660
Epoch 5/20
2740/2740 [==============================] - 313s 114ms/step - loss: 0.2585 - accuracy: 0.9204 - val_loss: 0.5879 - val_accuracy: 0.7660
Epoch 6/20
2740/2740 [==============================] - 304s 111ms/step - loss: 0.2582 - accuracy: 0.9205 - val_loss: 0.5613 - val_accuracy: 0.7660
Epoch 7/20
2740/2740 [==============================] - 252s 92ms/step - loss: 0.2575 - accuracy: 0.9205 - val_lo

In [35]:
save_model(model_GRU_REDUCED, 'GRU', 'BIN_REDUCED')

INFO:tensorflow:Assets written to: ram://3eff3c0e-91ee-4c08-9478-0da73dc87297/assets


INFO:tensorflow:Assets written to: ram://3eff3c0e-91ee-4c08-9478-0da73dc87297/assets


In [17]:
model_GRU_REDUCED = load_model('./GRU_BIN_REDUCED.sav')

In [18]:
gru_reduced_bin = model_GRU_REDUCED.predict(x_test_reduced_bin)

In [19]:
pre5 = [int(round(i,0)) for i in gru_reduced_bin.flatten()]

In [21]:
accuracy3 = accuracy_score(y_test_reduced_bin, pre5)
precision3 = precision_score(y_test_reduced_bin, pre5, average='binary')
recall3 = recall_score(y_test_reduced_bin, pre5, average='binary')
f1_3 = f1_score(y_test_reduced_bin, pre5, average='binary')

print(accuracy3, precision3, recall3, f1_3)

0.8236165767866687 0.7587683934690587 0.9964484249536751 0.8615158681721086


In [ ]:
y_train_multi = df_training['attack_cat']
y_test_multi = df_testing['attack_cat']

attack_categories = {name: index for index, name in enumerate(df_training['attack_cat'].unique())}
for category in attack_categories:
    y_train_multi = df_training['attack_cat'].replace(category, attack_categories[category])
    
for category in attack_categories:
    y_test_multi = df_testing['attack_cat'].replace(category, attack_categories[category])
    

y_train_multi = y_train_multi.to_numpy()
y_test_multi = y_test_multi.to_numpy()

# remove nan values
y_train_multi = remove_1d_array_nan_values(y_train_multi, attack_categories)
y_test_multi = remove_1d_array_nan_values(y_test_multi, attack_categories)
y_train_multi = y_train_multi.astype('int')
y_test_multi = y_test_multi.astype('int')

In [ ]:
model_LSTM_BIN_ALL = Sequential()

model_LSTM_BIN_ALL.add(LSTM(300, activation="tanh", return_sequences = True, input_shape = (42, 1)))
model_LSTM_BIN_ALL.add(Dropout(0.4))

model_LSTM_BIN_ALL.add(LSTM(200, activation="tanh", return_sequences = True))
model_LSTM_BIN_ALL.add(Dropout(0.4))

model_LSTM_BIN_ALL.add(LSTM(100, activation="tanh", return_sequences = True))
model_LSTM_BIN_ALL.add(Dropout(0.4))

model_LSTM_BIN_ALL.add(LSTM(80, activation="tanh"))
model_LSTM_BIN_ALL.add(Dropout(0.4))

model_LSTM_BIN_ALL.add(Dense(1))
model_LSTM_BIN_ALL.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model_LSTM_BIN_ALL.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_LSTM_BIN_ALL.fit(x_train_multi, y_train_multi, batch_size=64, 
          epochs=20, validation_data=(x_test, y_test_multi))

In [ ]:
save_model(model_LSTM_BIN_ALL, 'LSTM', 'MULTI_ALL')

In [ ]:
predict_lstm_multi_all = model_LSTM_BIN_ALL.predict(x_test)

In [ ]:
pre6 = [int(round(i,0)) for i in predict_lstm_multi_all.flatten()]

In [ ]:
accuracy4 = accuracy_score(y_test_multi, pre6)
precision4 = precision_score(y_test_multi, pre6, average='binary')
recall4 = recall_score(y_test_multi, pre6, average='binary')
f1_4 = f1_score(y_test_multi, pre6, average='binary')

print(accuracy4, precision4, recall4, f1_4)

In [ ]:
model_GRU_MULTI = Sequential()

model_GRU_MULTI.add(GRU(300, activation="tanh", return_sequences = True, input_shape = (20, 1)))
model_GRU_MULTI.add(Dropout(0.4))

model_GRU_MULTI.add(GRU(200, activation="tanh", return_sequences = True))
model_GRU_MULTI.add(Dropout(0.4))

model_GRU_MULTI.add(GRU(100, activation="tanh", return_sequences = True))
model_GRU_MULTI.add(Dropout(0.4))

model_GRU_MULTI.add(GRU(80, activation="tanh"))
model_GRU_MULTI.add(Dropout(0.4))

model_GRU_MULTI.add(Dense(1))
model_GRU_MULTI.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model_GRU_MULTI.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_GRU_MULTI.fit(x_train_multi, y_train_multi, batch_size=64, 
          epochs=20, validation_data=(x_test, y_test_multi))

In [ ]:
save_model(model_GRU_MULTI, 'GRU', 'MULTI_ALL')

In [ ]:
predict_gru_multi_all = model_GRU_MULTI.predict(x_test)

In [ ]:
pre7 = [int(round(i,0)) for i in predict_gru_multi_all.flatten()]

In [ ]:
accuracy4 = accuracy_score(y_test_multi, pre7)
precision4 = precision_score(y_test_multi, pre7, average='binary')
recall4 = recall_score(y_test_multi, pre7, average='binary')
f1_4 = f1_score(y_test_multi, pre7, average='binary')

print(accuracy4, precision4, recall4, f1_4)

In [ ]:
model_LSTM_MULTI_REDUCED = Sequential()

model_LSTM_MULTI_REDUCED.add(LSTM(300, activation="tanh", return_sequences = True, input_shape = (20, 1)))
model_LSTM_MULTI_REDUCED.add(Dropout(0.4))

model_LSTM_MULTI_REDUCED.add(LSTM(200, activation="tanh", return_sequences = True))
model_LSTM_MULTI_REDUCED.add(Dropout(0.4))

model_LSTM_MULTI_REDUCED.add(LSTM(100, activation="tanh", return_sequences = True))
model_LSTM_MULTI_REDUCED.add(Dropout(0.4))

model_LSTM_MULTI_REDUCED.add(LSTM(80, activation="tanh"))
model_LSTM_MULTI_REDUCED.add(Dropout(0.4))

model_LSTM_MULTI_REDUCED.add(Dense(1))
model_LSTM_MULTI_REDUCED.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model_LSTM_MULTI_REDUCED.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])